In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix , classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# =============================
# 🔹 Load Dataset
# =============================
file_path = 'dataset_sdn.csv'
df = pd.read_csv(file_path, encoding='latin1')

# Identify numeric and categorical features
numerical_features = [feature for feature in df.columns if df[feature].dtypes != 'O' and feature != "label"]
print("The number of numerical features is",len(numerical_features),"and they are : \n",numerical_features)
categorical_features = [feature for feature in df.columns if df[feature].dtypes == 'O']
print("The number of categorical features is",len(categorical_features),"and they are : \n",categorical_features)
target = 'label'

# Separate features and target variable
X = df[numerical_features + categorical_features]
y = df[target]

# Split into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# =============================
# 🔹 Handle Missing Values
# =============================
# Impute missing values in numeric features
num_imputer = SimpleImputer(strategy='mean')
X_train_numeric = num_imputer.fit_transform(X_train[numerical_features])
X_test_numeric = num_imputer.transform(X_test[numerical_features])

# Impute missing values in categorical features
cat_imputer = SimpleImputer(strategy='most_frequent')
X_train_categorical = cat_imputer.fit_transform(X_train[categorical_features])
X_test_categorical = cat_imputer.transform(X_test[categorical_features])

# =============================
# 🔹 Standardize Numeric Features
# =============================
scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

# =============================
# 🔹 Encode Categorical Features
# =============================
vectorizer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat_encoded = vectorizer.fit_transform(X_train_categorical)
X_test_cat_encoded = vectorizer.transform(X_test_categorical)

# =============================
# 🔹 Combine Processed Features
# =============================
X_train_final = np.hstack((X_train_numeric_scaled, X_train_cat_encoded))
X_test_final = np.hstack((X_test_numeric_scaled, X_test_cat_encoded))

# Ensure target values match indices after processing
y_train = y_train.iloc[:X_train_final.shape[0]].values  # Convert to NumPy array
y_test = y_test.iloc[:X_test_final.shape[0]].values     # Convert to NumPy array

# =============================
# 🔹 Train Support Vector Machine (SVM) Model
# =============================
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, random_state=42)

# Train the model
svm_model.fit(X_train_final, y_train)

# =============================
# 🔹 Make Predictions & Evaluate
# =============================
y_pred = svm_model.predict(X_test_final)

# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of SVM Classifier: {accuracy:.4f}")
print(classification_report(y_test, y_pred))

# 🎯 Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
TN, FP, FN, TP = conf_matrix.ravel()
print(f"TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")

# =============================
# 🔹 Save Model, Scaler & Vectorizer Separately
# =============================
joblib.dump(svm_model, 'Modells/SVM_ddos_model.pkl')
joblib.dump(scaler, 'Modells/SVM_ddos_scaler.pkl')
joblib.dump(vectorizer, 'Modells/SVM_ddos_vectorizer.pkl')

print("SVM model, scaler, and vectorizer saved separately.")

The number of numerical features is 19 and they are : 
 ['dt', 'switch', 'pktcount', 'bytecount', 'dur', 'dur_nsec', 'tot_dur', 'flows', 'packetins', 'pktperflow', 'byteperflow', 'pktrate', 'Pairflow', 'port_no', 'tx_bytes', 'rx_bytes', 'tx_kbps', 'rx_kbps', 'tot_kbps']
The number of categorical features is 3 and they are : 
 ['src', 'dst', 'Protocol']
Accuracy of SVM Classifier: 0.9866
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     12712
           1       0.98      0.99      0.98      8157

    accuracy                           0.99     20869
   macro avg       0.99      0.99      0.99     20869
weighted avg       0.99      0.99      0.99     20869

TP: 8051, TN: 12539, FP: 173, FN: 106
SVM model, scaler, and vectorizer saved separately.
